# Transformer 구조의 이해
<center>

![](./img/transformer.png)
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "fig 1: Transformer Model"
</figcaption>
  
</center>

위 fig 1 은, Attention is all you need 논문에서 제안된 transformer 모델의 architecture 입니다.\
encoder & decoder 구조로, 많은 분야에 큰 영향을 주었죠.

특히, transformer 의 꽃이라고 말 할 수 있는 부분은 Attention 구조입니다.\
해당 구조를 통해서 많은 성능 개선과 아이디어들이 등장할 수 있게 되었죠. 

다양한 논문들이 Transformer 를 변형하여 새로운 task 를 다루게 되었습니다.\
대표적으로 다음과 같은 변형 방법이 있습니다. 

- Transformer 의 encoder 만을 사용하는 방법 
- Transformer 의 decoder 만을 사용하는 방법

Transformer 의 encoder 만을 사용하는 방법으로 대표적인 예시는 BERT (bidirectional Encoder Representations from Transformers) 가 있겠네요.\
또한, **ViT 도 encoder 만을 사용하는 방법**입니다.

Transformer 의 decoder 만을 사용하는 방법으로 대표적인 예시는 GPT 입니다. 


><details><summary>
>
>### BERT & GPT
></summary>
> BERT 와 GPT 는 모두 Transformer 의 일부 형태를 사용합니다.
>
> 다만, 모델의 구조만을 보면 서로 동일하다고 생각하기 쉽습니다.
>
> 이 두 모델을 정확하게 이해하려면, 각 모델이 **학습하는 과정**과 **inference 과정**을 비교해 보는 것이 좋습니다. 
><center>
> 
> ![](./img/BertAndGpt.png)
> <figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
>   fig 2: BERT & GPT 
>    출처: Book [Foundation Models for Natural Language Processing Pre-trained Language Models Integrating Media]
>   </figcaption>
>   
> </center>
>
> #### BERT
> BERT 는 전체 시퀀스를 한번에 입력 받습니다. 
> 다만, 특정 문장을 [MASK] 처리하여 해당 [MASK] 에 들어갈 문장을 학습하고, 추론합니다. 
>
> #### GPT
> GPT 는 앞선 문장들을 입력으로 받으며, 다음에 등장할 단어를 추론하는 방식으로 학습합니다. 
> 그러므로, [the mouse eats cheese] 라는 문장을 아래와 같이 여러번의 입력으로 나누고 recursive 하게 다음 단어를 추론하게 됩니다.
> - the
> - the mouse
> - the mouse eats 
> - ...
> 
></details>


이전 장에서 ViT 의 입력으로 사용되는 이미지를 patch 단위로 나누었고, vector 로 embedding 했습니다. 

이제, Transformer 의 encoder 를 구현하기 위한 Multi-Head Attention 을 알아보죠!

----------
# Multi-Head Attention
<center>

![](./img/multi_head_attention.png)
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "fig 3: multi-head-attention"
</figcaption>
  
</center>

MHA는 위 그림과 같이 진행됩니다. VIT에서의 MHA는 QKV가 모두 동일한 embedding vector 입니다.\
입력텐서는 head 개의 Linear Projection 을 통해 임베딩된 후,각각 Scaled Dot-Product Attention을 진행합니다.

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from collections import OrderedDict
from typing import Optional

from utils.vit_utils import Image_Embedding # 이전 장의 image embedding
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
ims = torch.Tensor(np.load('./resources/test_images.npy', allow_pickle=False))
ims = rearrange(ims, 'b h w c -> b c h w')
print(type(ims), ims.shape)

<class 'torch.Tensor'> torch.Size([6, 3, 96, 96])


In [3]:
image_embedding = Image_Embedding(image_size = ims.shape, patch_size=16).to(device)
embedded_tensor = image_embedding(ims.to(device))
summary(image_embedding, ims.shape[1:])
print('Output shape: {}'.format(embedded_tensor.shape))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         LayerNorm-1            [-1, 3, 96, 96]          55,296
            Conv2d-2            [-1, 768, 6, 6]         590,592
         LayerNorm-3            [-1, 768, 6, 6]          55,296
Total params: 701,184
Trainable params: 701,184
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.11
Forward/backward pass size (MB): 0.63
Params size (MB): 2.67
Estimated Total Size (MB): 3.41
----------------------------------------------------------------
Output shape: torch.Size([6, 37, 768])


위와 같이 6장의 이미지는 [6, 37, 768]크기로 embedding 된 것을 알 수 있습니다.\
이제 이 tensor 를 가지고 Multi Head Attention 을 구현해 보죠.

In [4]:
embedding_size = embedded_tensor.shape[-1]
num_heads = 8

K = nn.Linear(embedding_size, embedding_size).to(device)
Q = nn.Linear(embedding_size, embedding_size).to(device)
V = nn.Linear(embedding_size, embedding_size).to(device)

keys    = rearrange(K(embedded_tensor), "b n (h d) -> b h n d", h=num_heads)
queries = rearrange(Q(embedded_tensor), "b n (h d) -> b h n d", h=num_heads)
values  = rearrange(V(embedded_tensor), "b n (h d) -> b h n d", h=num_heads)

print('''keys shape    : {}
queries shape : {}
values        : {}'''.format(keys.shape, queries.shape, values.shape))

keys shape    : torch.Size([6, 8, 37, 96])
queries shape : torch.Size([6, 8, 37, 96])
values        : torch.Size([6, 8, 37, 96])


각 차원을 다시 한 번 점검해 보도록 합시다. 
```python
torch.Size([    6,    8,    37,             96])
torch.Size([Batch, Head, Patch, Embedding size])
```

이제는 Scaled dot-product attention 을 구현해 봅시다.

-------

## Scaled dot-product attention

<center>
<figure>
    <img src="./img/Scaled_dot_product_attention.png" alt="Scaled dot-product attention" width="50%" height="50%">
</figure>
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "fig 3: Scaled dot-product attention"
</figcaption>

</center>

위 그림을 수식화 하면 다음과 같습니다. 

$$
\text{Attention}(Q, K, V) = \text{softmax} \left( \frac{QK^T}{\sqrt{d_k}} \right) V
$$

코드로 구현해 보죠!

In [5]:
# QK^T
QK = torch.einsum('b h q d, b h k d -> b h q k', queries, keys)
print('QK :', QK.shape)

QK : torch.Size([6, 8, 37, 37])


Einops 의 사용 방법에 어느 정도 익숙해 졌으리라 믿습니다.

그러므로 위 문법도 내적의 관점에서 잘 이해해 보세요!

In [6]:
# Attention Score
scaling = embedding_size ** (1/2)
attention_score = F.softmax(QK/scaling, dim=-1)
print('Shape of Attention Score :', attention_score.shape)

Shape of Attention Score : torch.Size([6, 8, 37, 37])


위의, Attention Score 의 차원에 대해서 다시 정리해 보죠.

```python
torch.Size([    6,    8,    37,             37])
torch.Size([Batch, Head, Patch, Patch])
```

즉, 각 Patch 들 끼리의 Attention Score 라는 것을 잊지 마세요.

In [7]:
# Attention Score * values
representation = torch.einsum('b h p d, b h d v -> b h p v ', attention_score, values)
print('Shape of Representation :', representation.shape)

Shape of Representation : torch.Size([6, 8, 37, 96])


이렇게 만들어진 Attention Score 를 values 와 내적하여,\
**"Attention Score 가 반영된 Representation"** 을 구합니다. 

이후, head 로 나누었던 차원을 다시 복원해 주도록 하죠.

In [8]:
concated = rearrange(representation, "b h p d -> b p (h d)")
print('Shape of Concated : ', concated.shape)

Shape of Concated :  torch.Size([6, 37, 768])


In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, 
                 embedding_size: int = 768, 
                 num_heads: int = 8, 
                 dropout: float = 0):
        super(MultiHeadAttention, self).__init__()
        
        self.embedding_size = embedding_size
        self.num_heads = num_heads
        
        # keys, queries, values
        self.K = nn.Linear(embedding_size, embedding_size)
        self.Q = nn.Linear(embedding_size, embedding_size)
        self.V = nn.Linear(embedding_size, embedding_size)

        # drop out
        self.att_drop = nn.Dropout(dropout)
        
        self.projection = nn.Linear(embedding_size, embedding_size)
        
    def forward(self, x : Tensor) -> Tensor:
        # keys, queries, values
        keys    = rearrange(self.K(embedded_tensor), "b n (h d) -> b h n d", h=self.num_heads)
        queries = rearrange(self.Q(embedded_tensor), "b n (h d) -> b h n d", h=self.num_heads)
        values  = rearrange(self.V(embedded_tensor), "b n (h d) -> b h n d", h=self.num_heads)

        # Attention Score
        QK = torch.einsum('b h q d, b h k d -> b h q k', queries, keys)
        scaling = self.embedding_size ** (1/2)
        attention_score = F.softmax(QK/scaling, dim=-1)
        representation = torch.einsum('b h p d, b h d v -> b h p v ', attention_score, values)

        # Concat and projection
        concated = rearrange(representation, "b h p d -> b p (h d)")
        out = self.projection(concated)

        return out

In [10]:
# 지난 장
image_embedding = Image_Embedding(image_size = ims.shape, patch_size=16).to(device)
embedded_tensor = image_embedding(ims.to(device))


mha = MultiHeadAttention(embedded_tensor.shape[-1], 8, 0).to(device)
mha_pass = mha(embedded_tensor)
summary(mha, embedded_tensor.shape[1:])
print('Output shape: {}'.format(mha_pass.shape))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 37, 768]         590,592
            Linear-2              [-1, 37, 768]         590,592
            Linear-3              [-1, 37, 768]         590,592
            Linear-4              [-1, 37, 768]         590,592
Total params: 2,362,368
Trainable params: 2,362,368
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.11
Forward/backward pass size (MB): 0.87
Params size (MB): 9.01
Estimated Total Size (MB): 9.99
----------------------------------------------------------------
Output shape: torch.Size([6, 37, 768])


------

# 정리 

여기까지가 Multi-Head Attention 을 구현한 것 입니다. 

이제 다음장에서는, fig 1 에 나와있는 나머지 부분을 구현해 보도록 하죠!

Residual connection 과, feed forward, add norm 이 남아 있습니다.